Developer name: Yvar Joseph

Project: Summarizer Model for Healthcare Documentation

In [ ]:
!pip install evaluate #To evaluate various pre-trained models on our dataset
!pip install rouge #To evaluate the rouge metrics for our summarization models
!pip install sacrebleu #To evaluate the sacrebleu metrics for our summarization models

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import numpy as np
import pandas as pd

Loading the dataset for evaluation

In [ ]:
#Import
import datasets
from datasets import load_dataset

In [ ]:
pubmed_ds = load_dataset("ronitHF/pubmed-10k-8.1.1")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/484 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/188M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/23.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [ ]:
pubmed_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 1250
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 1250
    })
})

In [ ]:
#Performing data splits
pubmed_ds_train = pubmed_ds["train"]
pubmed_ds_valid = pubmed_ds["validation"]

In [ ]:
#Evaluating the columns of our dataset and their respective lengths
print(f'Dataset features: {pubmed_ds_train.column_names}')
print(f'Length of our training set: {len(pubmed_ds_train)}')
print(f'Length of our validation set: {len(pubmed_ds_valid)}')

Dataset features: ['article', 'abstract', 'section_names']
Length of our training set: 10000
Length of our validation set: 1250


In [ ]:
#Peeking into the first 500 characters of the training set's articles
sample_article_iter = pubmed_ds_train["article"][0][:500]
sample_asbtract_iter = pubmed_ds_train["abstract"][0]
sample_article, sample_abstract = sample_article_iter, sample_asbtract_iter
print(f"""
SAMPLE ARTICLE AND ABSTRACTION FOR THE PUBMED DATASET

First 500 characters of sample article | Total Size: {len(pubmed_ds_train["article"][0])}
===============================================================================
{sample_article}
===============================================================================
Sample Summary

{sample_abstract}
""")


SAMPLE ARTICLE AND ABSTRACTION FOR THE PUBMED DATASET

First 500 characters of sample article | Total Size: 28156
a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .
in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .
the prevalence of malnutrition among elementary school aged childr
Sample Summary

 background : the present study was carried out to assess the effects of community nutrition intervention based on advocacy approach on malnutrition status among school - aged children in shiraz , iran.materials and methods : this case - control nutritional intervention has been done between 2008 and 2009 on 2897 primary and secondary school boys and girls ( 7 - 13 

Phase 2 - Preprocessing the dataset prior to tokenization


In [ ]:
from datasets import Dataset
#For a clean preprocessing of the dataset, the Dataset object will be converted to a pandas Dataframe
print(f'Dataset object type', type(pubmed_ds_train))
#Converting the Dataset object to a pandas dataframe
pubmed_df = pubmed_ds_train.to_pandas()
#Evaluate the first five rows of data
pubmed_df.head()

Dataset object type <class 'datasets.arrow_dataset.Dataset'>


,article,abstract,section_names
0,a recent systematic analysis showed that in 20...,background : the present study was carried ou...,INTRODUCTION\nMATERIALS AND METHODS\nParticipa...
1,it occurs in more than 50% of patients and may...,backgroundanemia in patients with cancer who ...,Introduction\nPatients and methods\nStudy desi...
2,"tardive dystonia ( td ) , a rarer side effect ...",tardive dystonia ( td ) is a serious side eff...,INTRODUCTION\nCASE REPORT\nDISCUSSION\nDeclara...
3,"lepidoptera include agricultural pests that , ...",many lepidopteran insects are agricultural pe...,1. Introduction\n2. Insect Immunity\n3. Signal...
4,syncope is caused by transient diffuse cerebra...,we present an unusual case of recurrent cough...,Introduction\nCase report\nDiscussion\nConflic...


In [ ]:
pubmed_df_valid = pubmed_ds_valid.to_pandas()

In [ ]:
pubmed_df.columns

Index(['article', 'abstract', 'section_names'], dtype='object')

In [ ]:
#Dropping undeed columns
pubmed_df.drop(columns=["section_names"], inplace=True)

In [ ]:
pubmed_df_valid.drop(columns=["section_names"], inplace=True)

In [ ]:
pubmed_df.head()

,article,abstract
0,a recent systematic analysis showed that in 20...,background : the present study was carried ou...
1,it occurs in more than 50% of patients and may...,backgroundanemia in patients with cancer who ...
2,"tardive dystonia ( td ) , a rarer side effect ...",tardive dystonia ( td ) is a serious side eff...
3,"lepidoptera include agricultural pests that , ...",many lepidopteran insects are agricultural pe...
4,syncope is caused by transient diffuse cerebra...,we present an unusual case of recurrent cough...


In [ ]:
#Checking the dataframe for null values
check_nulls = lambda x: x.isnull().sum() > 0
pubmed_df.apply(check_nulls)

,0
article,True
abstract,False


In [ ]:
check_nulls_valid = lambda x: x.isnull().sum() > 0
pubmed_df_valid.apply(check_nulls_valid)

,0
article,False
abstract,False


In [ ]:
#Since the validation dataset has no null values, we can return it to its dataset format
from datasets import Dataset
pubmed_ds_valid = Dataset.from_pandas(pubmed_df_valid)

In [ ]:
#Removing nan rows
pubmed_df.dropna(inplace=True)

In [ ]:
len(pubmed_df)

9739

In [ ]:
#Analyzing the first 3000 words of a sample article
sample_article = pubmed_df["article"][1][:3000]
sample_article

'it occurs in more than 50% of patients and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy.1 anemia is defined as an inadequate circulating level of hemoglobin ( hb ) ( hb < 12 g / dl ) and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy .\npreliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck cancers , may be compromised by anemia.24 anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life ( qol ) and performance status in patients with cancer .\nthus , to improve physical functioning , qol , and prognosis in patients with cancer , it would be reasonable to take a proactive approach in identifying populations who need treatment for cancer - associated anemia ( caa ) and provide timely management .\nblood transfusi

In [ ]:
import nltk
import re

def preprocessor(dataset):
    dataset = dataset.lower()
    dataset = re.sub(r'(background\s+:)|(conclusion\s+:)|(\n)', '', dataset)
    dataset = re.sub(r'\(\s*\w+(\s+\w+)*\s*\)|\(\w+(\s+.|\S)\d\)', '', dataset)
    dataset = re.sub(r'\([^)]*\)', '', dataset)
    dataset = re.sub(r'[\?\:\!\;\[\]\*\<\>]', '', dataset)
    dataset = re.sub(r'\.(\d+,\d+|\d+)', '.', dataset)
    dataset = dataset.replace('.', ' . ')
    dataset = dataset.replace('%', ' % ')
    found_pattern = re.findall(r'(\w+\s+/\s+\w+\s+)', dataset)[:]
    for pattern in found_pattern:
        if pattern in dataset:
           pattern_updated = pattern.replace('/', 'per')
           dataset = dataset.replace(pattern, pattern_updated)

    return dataset.strip()

In [ ]:
preprocessor(sample_article) #Testing the dummy preprocessor function

'a recent systematic analysis showed that in 2011 , 314  million children younger than 5 years were mildly , moderately or severely stunted and 258  million were mildly , moderately or severely underweight in the developing countries  . in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16 .  %  , 8 .  %  and 1 .  %  , for underweight , overweight and obesity , respectively  . the prevalence of malnutrition among elementary school aged childr'

In [ ]:
#Developing a preprocessor function
import re
def preprocessor(dataset, feature="article"):
    dataset[feature] = dataset[feature].lower()
    dataset[feature] = re.sub(r'(background\s+:)|(conclusion\s+:)|(\n)', '', dataset[feature])
    dataset[feature] = re.sub(r'\(\s*\w+(\s+\w+)*\s*\)|\(\w+(\s+.|\S)\d\)', '', dataset[feature])
    dataset[feature] = re.sub(r'\([^)]*\)', '', dataset[feature])
    dataset[feature] = re.sub(r'[\?\:\!\;\[\]\*\<\>]', '', dataset[feature])
    dataset[feature] = re.sub(r'\.(\d+,\d+|\d+)', '.', dataset[feature])
    dataset[feature] = dataset[feature].replace('.', ' . ')
    dataset[feature] = dataset[feature].replace('%', ' % ')
    found_pattern = re.findall(r'(\w+\s+/\s+\w+\s+)', dataset[feature])[:]
    for pattern in found_pattern:
        if pattern in dataset[feature]:
           pattern_updated = pattern.replace('/', 'per')
           dataset[feature] = dataset[feature].replace(pattern, pattern_updated)
    return dataset

In [ ]:
preprocessor(pubmed_ds["train"][1])["article"][:2000] #Testing the preprocessor on a sampl

'it occurs in more than 50 %  of patients and may reach 90 %  in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy .  anemia is defined as an inadequate circulating level of hemoglobin   and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy  . preliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck cancers , may be compromised by anemia .  anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life  and performance status in patients with cancer  . thus , to improve physical functioning , qol , and prognosis in patients with cancer , it would be reasonable to take a proactive approach in identifying populations who need treatment for cancer - associated anemia  and provide timely management  . blood transfusion is an effective way to replace

In [ ]:
# #Reducing the size of the dataset to 30% for training purposes
# import math
# from datasets import Dataset

# pubmed_data_train_mini = pubmed_ds[:math.floor(len(pubmed_ds["article"]) * 0.30)]
# print(len(pubmed_data_train_mini))
# #Convert this reduced dataframe to a dataset again
# pubmed_ds = Dataset.from_pandas(pubmed_data_train_mini)
# pubmed_ds

In [ ]:
#Convert dataframe back to dataset for preprocessing implementation
from datasets import Dataset

updated_ds = Dataset.from_pandas(pubmed_df)

In [ ]:
updated_ds["article"][1][:2000]

'it occurs in more than 50% of patients and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy.1 anemia is defined as an inadequate circulating level of hemoglobin ( hb ) ( hb < 12 g / dl ) and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy .\npreliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck cancers , may be compromised by anemia.24 anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life ( qol ) and performance status in patients with cancer .\nthus , to improve physical functioning , qol , and prognosis in patients with cancer , it would be reasonable to take a proactive approach in identifying populations who need treatment for cancer - associated anemia ( caa ) and provide timely management .\nblood transfusi

In [ ]:
#Applying the preprocessor to the overall dataset
updated_ds = updated_ds.map(lambda x: preprocessor(x, feature="article"))
updated_ds = updated_ds.map(lambda x: preprocessor(x, feature="abstract"))

Map:   0%|          | 0/9739 [00:00<?, ? examples/s]

Map:   0%|          | 0/9739 [00:00<?, ? examples/s]

In [ ]:
updated_ds["abstract"][1][:900] #testing the preprocessed data

' backgroundanemia in patients with cancer who are undergoing active therapy is commonly encountered and may worsen quality of life in these patients  .  the effect of blood transfusion is often temporary and may be associated with serious adverse events  .   erythropoiesis - stimulating agents are not effective in 30 % 50 %  of patients and may have a negative effect on overall survival . aimsto assess the efficacy and feasibility of intravenous iron therapy in patients with cancer who have non - iron - deficiency anemia and who are undergoing treatment with chemotherapy without the use of erythropoiesis - stimulating agents . methodsadult patients with solid cancers and non - iron - deficiency anemia were included  .   ferric sucrose at a dose of 200 mg was given in short intravenous infusions weekly for a total of 12 weeks  .   hemoglobin level was measured at baseline , every 3 weeks '

In [ ]:
pubmed_ds_valid = pubmed_ds_valid.map(lambda x: preprocessor(x, feature="article"))
pubmed_ds_valid = pubmed_ds_valid.map(lambda x: preprocessor(x, feature="abstract"))

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [ ]:
print(pubmed_ds_valid["article"][1][:500])
print("\n\nUpdated abstract\n")
print(pubmed_ds_valid["abstract"][1])

there is an epidemic of stroke in low and middle - income countries due to rapidly increasing prevalence of vascular risk factors such as hypertension , diabetes mellitus , dyslipidaemia  . the need has thus arisen to conduct studies aimed at epidemiologic , genetic and phenotypic characterisation of stroke in sub - saharan africa to provide evidence - based information to confront this menace  .  to this end, it is important that controls selected for comparison with cases are recruited with a 


Updated abstract

 backgroundthe questionnaire for verifying stroke - free status  has been validated in western populations as a method for verifying stroke - free status in participants of clinical , epidemiological and genetic studies  .   this instrument has not been validated in low - income settings where populations have limited knowledge of stroke symptoms and literacy levels are low . objectiveto simultaneously validate the 8-item qvsfs in 3 languages spoken in west africa  for ascer

Phase 3 - Now that the dataset has been preprocessed as best as possible, the next phase will focus on building a baseline
for the various pre-trained models we will be using for accuracy testing. For sample testing, we will make use of two models:

1) Google's Pegasus

2) Facebook's T5 transformer model

In [ ]:
sample_text = updated_ds["article"][1][:2000]
#Adding the various model summaries in a dictionary
summaries = {}

In [ ]:
#Using the NLP convetion: seperating the summaries by a newline using NLTK
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt_tab")

#Testing the string
test_string = "The U.S. are a country. The U.N. is an organization"
sent_tokenize(test_string)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


['The U.S. are a country.', 'The U.N. is an organization']

For the medical dataset analysis, we will be using a baseline for summarization: using the first four sentences of the document

In [ ]:
def four_sentence_summary(document):
  return "\n".join(sent_tokenize(document)[:3])
summaries["baseline"] = four_sentence_summary(sample_text) #testing the baseline with the test (sample) string

In [ ]:
summaries

{'baseline': 'it occurs in more than 50 %  of patients and may reach 90 %  in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy .\nanemia is defined as an inadequate circulating level of hemoglobin   and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy  .\npreliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck cancers , may be compromised by anemia .'}

PEGASUS MODEL TESTING

In [ ]:
from transformers import pipeline

pegasus_checkpoint = "google/pegasus-xsum"
pipe = pipeline("summarization", model=pegasus_checkpoint)
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace("  .<n>", ".\n")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
summaries["pegasus"]

'Blood transfusions are an important part of the care of patients with cancer, but they are often used as a last resort for patients with low circulating hb levels .'

BART MODEL TESTING

In [ ]:
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
summaries["bart"]

'Anemia is defined as an inadequate circulating level of hemoglobin.\nit occurs in more than 50 %  of patients and may reach 90 % in certain types of cancers.\nBlood transfusion is an effective way to replace depleted hb within a short period.\nHowever, the effect is temporary and can cause serious adverse risks and increased mortality.'

T5 MODEL TESTING

In [ ]:
# pipe = pipeline("summarization", model="t5-large")
# pipe_out = pipe(sample_text)
# summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [ ]:
# summaries["t5"]

Phase 4 - Now that we have attained each of the respective summaries of our models, we can now compare their respective results

From the results above, it can be seen that BART seems to perform the best with respect to producing the most accurate summaries.

In [ ]:
print("GROUND TRUTH")
print(updated_ds["abstract"][1])
print("")

for model_name in summaries:
  print(model_name.upper())
  print(summaries[model_name])
  print("")


GROUND TRUTH
 backgroundanemia in patients with cancer who are undergoing active therapy is commonly encountered and may worsen quality of life in these patients  .  the effect of blood transfusion is often temporary and may be associated with serious adverse events  .   erythropoiesis - stimulating agents are not effective in 30 % 50 %  of patients and may have a negative effect on overall survival . aimsto assess the efficacy and feasibility of intravenous iron therapy in patients with cancer who have non - iron - deficiency anemia and who are undergoing treatment with chemotherapy without the use of erythropoiesis - stimulating agents . methodsadult patients with solid cancers and non - iron - deficiency anemia were included  .   ferric sucrose at a dose of 200 mg was given in short intravenous infusions weekly for a total of 12 weeks  .   hemoglobin level was measured at baseline , every 3 weeks , and 2 weeks after the last iron infusion   .   adverse events related to intravenous 

Model Evaluation Phase

In [ ]:
#Measuring the quality of the generated text with BLEU
import evaluate

bleu_metric = evaluate.load("sacrebleu")

In [ ]:
import pandas as pd
import numpy as np
#Performing a dummy prediction
bleu_metric.add(
    prediction="this is a dummy prediction",
    reference=["this dummy prediction"])
results = bleu_metric.compute(smooth_method="floor",
                              smooth_value=0)

results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,0.0
counts,"[3, 1, 0, 0]"
totals,"[5, 4, 3, 2]"
precisions,"[60.0, 25.0, 0.0, 0.0]"
bp,1.0
sys_len,5
ref_len,3


In [ ]:
#Dummy prediction 2
bleu_metric.add(
    predictions="the cat is on living room mat",
    references=["the cat is living room mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,0.0
counts,"[6, 4, 2, 0]"
totals,"[7, 6, 5, 4]"
precisions,"[85.71, 66.67, 40.0, 0.0]"
bp,1.0
sys_len,7
ref_len,6


Conclusion: We can see from the dummy predicition above, that
that our bleu score performs accurately on the 1-gram, less so
on the 2-gram, and is inneffective on 3 and 4 grams. Which makes sense given that the prediction and reference sentences do not match completely. This would somewhat mean that our model would be more precise on 1-grams (individual words) as opposed to multi-grams.

Performing calculations with ROUGE

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1d3d0e29fd8025544be010df5aa331ae3d3305c0e455f7f9c4cab52c37af9474
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
rouge_metric = evaluate.load("rouge")
reference = updated_ds["abstract"][1]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
  rouge_metric.add(prediction=summaries[model_name], reference=reference)
  score = rouge_metric.compute()
  rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
  records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.217765,0.051873,0.126074,0.120344


Evaluating the [Selected Model] on the PubMed Dataset

In [ ]:
def evulauate_summaries_baseline(dataset, metric,
                                 column_text="article",
                                 column_summary="abstract"):
  summaries= [four_sentence_summary(text) for text in dataset[column_text]]
  print(summaries)
  metric.add_batch(predictions=summaries,
                   references=dataset[column_summary])
  score = metric.compute()
  return score

In [ ]:
test_sampled = pubmed_ds_valid.shuffle(seed=42).select(range(500))
score = evulauate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

['the fast li - ion conductor  with the nominal composition li7la3zr2o12  is receiving much  scientific attention since its discovery in 2007  .\nit has a garnet - based structure , and it occurs in at least  two structural modifications  .\nat room temperature , llzo is tetragonal  while the cubic modification  is stable above approximately 150 c  .', 'diabetic nephropathy progressively affecting up to a third of patients with diabetes mellitus , a resource - consuming complication of diabetes and is the most common cause of renal failure  .\nalthough we have better knowledge related to this complication , the intimate mechanisms leading to the development and progression of renal injury are not exactly clear  .\nhyperglycemia is responsible for the succession of diabetic nephropathy through oxidative stress , renal polyol formation , mitogen - activated protein kinases , protein kinase c , and accumulation of early and advanced glycosylation end products   .', 'though kidney transpla

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.252334,0.083147,0.162884,0.184559


In [ ]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_elements, batch_size):
  for i in range(0, len(list_elements), batch_size):
    yield list_elements[i : i + batch_size]

def evaluate_summaries_bart(dataset,
                            metric,
                            model,
                            tokenizer,
                            batch_size=16,
                            device=device,
                            column_text="article",
                            column_summary="abstract"):
  article_batches = list(chunks(dataset[column_text], batch_size))
  target_batches = list(chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm (
      zip(article_batches, target_batches), total=len(article_batches)):
      inputs = tokenizer(article_batch, max_length=1020,
                         truncation=True,
                         padding="max_length",
                         return_tensors="pt") #generate the sequences of our article batches
      summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                 attention_mask=inputs["attention_mask"].to(device),
                                 length_penalty=0.8,
                                 num_beams=8,
                                 max_length=300) #max generation length
      decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                            clean_up_tokenization_spaces=True)
                                            for s in summaries]


      #Performing the overall analysis with our metric (rouge_score)
      decoded_summaries = ["\n".join(sent_tokenize(d)) for d in decoded_summaries] # Corrected line
      metric.add_batch(predictions=decoded_summaries, references=target_batch)

  score = metric.compute()
  return score

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_checkpoint = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device) # Move model to CPU

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
test_sampled = pubmed_ds_valid.shuffle(seed=42).select(range(100))

In [ ]:
test_sampled

Dataset({
    features: ['article', 'abstract'],
    num_rows: 100
})

In [ ]:
score = evaluate_summaries_bart(test_sampled, rouge_metric,
                                model, tokenizer, batch_size=6)
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

100%|██████████| 17/17 [04:24<00:00, 15.55s/it]


In [ ]:
pd.DataFrame(rouge_dict, index=["bart"])

,rouge1,rouge2,rougeL,rougeLsum
bart,0.249943,0.082069,0.164486,0.187595


In [ ]:
# #Training our summarization model
# import matplotlib.pyplot as plt


# d_len = [len(tokenizer.encode(s)) for s in updated_ds["article"]]
# s_len = [len(tokenizer.encode(s)) for s in updated_ds["abstract"]]

# fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
# axes[0].hist(d_len, bins=20, color="C0", edgecolor="C0")
# axes[0].set_title("Dialogue Token Length")
# axes[0].set_xlabel("Length")
# axes[0].set_ylabel("Count")
# axes[1].hist(s_len, bins=20, color="C0", edgecolor="C0")
# axes[1].set_title("Summary Token Length")
# axes[1].set_xlabel("Length")
# plt.tight_layout()
# plt.show()

In [ ]:
#Initiating the model training
def convert_examples_to_sequences(batch):
  input_encodings = tokenizer(batch["article"],
                              max_length=1024,
                              truncation=True)

  target_encodings = tokenizer(text_target=batch["abstract"], #adds special tokens to the summarizers
                                 max_length=300,
                                 truncation=True)
  return {
      #collect the sequences
      "input_ids": input_encodings["input_ids"],
      "attention_mask": input_encodings["attention_mask"],
      "labels": target_encodings["input_ids"]
  }

dataset_pubmed = updated_ds.map(convert_examples_to_sequences,
                                  batched=True)
dataset_pubmed_eval = pubmed_ds_valid.map(convert_examples_to_sequences,
                                             batched=True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_pubmed.set_format(type="torch", columns=columns)

Map:   0%|          | 0/9739 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir='bart_pubmed_dataset_mark2',
    num_train_epochs=1,
    warmup_steps=400,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    push_to_hub=True,
    hub_model_id="Pubmed_Summarizer_Model_Trained",
    eval_strategy='steps',
    eval_steps=400,
    save_steps=1e6,
    predict_with_generate=True,
    gradient_accumulation_steps=16,
    report_to="none"
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer = Seq2SeqTrainer(model=model, args=training_args,
                  tokenizer=tokenizer,
                  data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pubmed,
                  eval_dataset=dataset_pubmed_eval,
                  )

/tmp/ipython-input-893596174.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(model=model, args=training_args,


In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
400,2.253500,2.092517


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3917: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=609, training_loss=2.2417168264905807, metrics={'train_runtime': 5679.9425, 'train_samples_per_second': 1.715, 'train_steps_per_second': 0.107, 'total_flos': 2.056989869305037e+16, 'train_loss': 2.2417168264905807, 'epoch': 1.0})

In [ ]:
pubmed_valid_mini = pubmed_ds_valid[ :round(len(pubmed_ds_valid) * 0.20)]

In [ ]:
pubmed_ds_valid_mini = pubmed_ds_valid.to_pandas()[:round(len(pubmed_ds_valid) * 0.10)]
pubmed_ds_valid_mini = Dataset.from_pandas(pubmed_ds_valid_mini)
pubmed_ds_valid_mini[0]["abstract"]

' background and aim  there is lack of substantial indian data on venous thromboembolism   .   the aim of this study was to provide real - world information on patient characteristics , management strategies , clinical outcomes , and temporal trends in vte . subjects and methods  multicentre retrospective registry involving 549 medical records of patients with confirmed diagnosis of vte  from 2006 to 2010 at three indian tertiary care hospitals . resultsacute dvt without pe , acute dvt with pe , and pe alone were reported in 64 %   , 23 %   , and 13 %   patients , respectively  .   mean age was 47  years , and 70 %  were males  .   h per o dvt  , surgery including orthopedic surgery  , trauma  , and immobilization  3 days  were the most common risk factors for vte  .   hypertension  , diabetes  , and neurological disease   were the most common co - morbidities  .   most  were treated with heparin alone  or fondaparinux  for initial anticoagulation  low molecular weight heparin alone  o

In [ ]:

score = evaluate_summaries_bart(pubmed_ds_valid_mini,
                                rouge_metric,
                                trainer.model,
                                tokenizer,
                                batch_size=2)

100%|██████████| 63/63 [16:23<00:00, 15.61s/it]


In [ ]:
score

{'rouge1': np.float64(0.3900288387474339),
 'rouge2': np.float64(0.15138421518186373),
 'rougeL': np.float64(0.2310912009410659),
 'rougeLsum': np.float64(0.2788017820125559)}

In [ ]:
sample_text_pred = """
His whole planet was destroyed. He's the last of a holocaust. He grew up in the dirt. Finding out slowly how different he was. A stranger discovering every day how strange he was. He has the power to tear the world apart. And he could. With a pinkie. It's not his world. We're not his people. We should be ants to him. Imagine that. Always being on the outside. The pain that would come from always being on the outside. And yet, he took that pain and became the symbol of hope

"""

In [ ]:
sequenced_text = tokenizer([pubmed_ds_valid[1]["article"]],
                            max_length=1024,
                            truncation=True,
                            return_tensors="pt")
summary = trainer.model.generate(sequenced_text["input_ids"].to(trainer.model.device),
                       attention_mask=sequenced_text["attention_mask"].to(trainer.model.device),
                       length_penalty=0.8,
                       num_beams=8,
                       max_length=128)
decoded_summary = tokenizer.decode(summary[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
decoded_summary

' this study seeks to validate the 8-item questionnaire for verifying stroke - free status  at 5 centers in nigeria and ghana which are involved in the on - going nih supported stroke investigative research and educational networks .   participants will be recruited from five tertiary referral medical centers in sub - saharan africa namely  west west university college, west west college, university college of nigeria, nigeria  ahmadu aminu bello university hospital, zaria university hospital  kano kano teaching hospital , kumasi - kano hospital  , pre - tested kano'

In [ ]:
#Push the model to the hub
trainer.push_to_hub("Pubmed_Summarizer_Model_Mark2")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...med_dataset_mark2/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...med_dataset_mark2/model.safetensors:   3%|2         | 41.9MB / 1.63GB            

CommitInfo(commit_url='https://huggingface.co/AIEngineerYvar/Pubmed_Summarizer_Model_Trained/commit/e6ff103f39d1829187e6a725560a0bbf9f60273e', commit_message='Pubmed_Summarizer_Model_Mark2', commit_description='', oid='e6ff103f39d1829187e6a725560a0bbf9f60273e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AIEngineerYvar/Pubmed_Summarizer_Model_Trained', endpoint='https://huggingface.co', repo_type='model', repo_id='AIEngineerYvar/Pubmed_Summarizer_Model_Trained'), pr_revision=None, pr_num=None)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("AIEngineerYvar/Pubmed_Summarizer_Model_Trained")
model = AutoModelForSeq2SeqLM.from_pretrained("AIEngineerYvar/Pubmed_Summarizer_Model_Trained").to(device)

/usr/local/lib/python3.12/dist-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
sequenced_text = tokenizer([sample_text_superman],
                            max_length=300,
                            truncation=True,
                            return_tensors="pt")
summary = model.generate(sequenced_text["input_ids"].to(model.device),
                       attention_mask=sequenced_text["attention_mask"].to(model.device),
                       length_penalty=0.8,
                       num_beams=8,
                       max_length=300)
decoded_summary = tokenizer.decode(summary[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
decoded_summary

'a System of Health Accounts 2011 provides an updated and systematic description of financial flows related to the consumption of health care goods and services related to the consumption of health care goods and services around the world. It builds on the original OECD Manual published in 2000 to create a single global framework for producing health expenditure accounts that can help track resource flows from sources to uses.'